# Stock Price Collector - Testing Notebook

This notebook tests the stock price collection job function by function.

## What we'll test:
1. Get top 50 tickers by article count
2. Fetch a single stock price
3. Validate price data
4. Update stock prices in database

In [1]:
import sys
import os
import asyncio
from datetime import datetime, timedelta, UTC

# Add project root to path
sys.path.insert(0, os.path.abspath('..'))

from app.db.session import SessionLocal
from app.db.models import Article, ArticleTicker, StockPrice, Ticker
from app.services.stock_data import StockDataService
from sqlalchemy import func

print("✅ Imports successful")

✅ Imports successful


## 1. Test: Get Top 50 Tickers

Query the database for the top 50 most active tickers based on article count in the last 24 hours.

In [2]:
def get_top_n_tickers(db, n: int = 50, hours: int = 24) -> list[str]:
    """
    Get top N most active tickers by article count in the last N hours.
    """
    cutoff_time = datetime.now(UTC) - timedelta(hours=hours)

    top_tickers = (
        db.query(
            ArticleTicker.ticker,
            func.count(ArticleTicker.article_id).label("article_count"),
        )
        .join(Article, ArticleTicker.article_id == Article.id)
        .filter(Article.published_at >= cutoff_time)
        .group_by(ArticleTicker.ticker)
        .order_by(func.count(ArticleTicker.article_id).desc())
        .limit(n)
        .all()
    )

    return [(ticker, count) for ticker, count in top_tickers]

# Test it
db = SessionLocal()
try:
    print("\n📊 Top 50 Tickers (last 24 hours):\n")
    top_tickers = get_top_n_tickers(db, n=50, hours=24)
    
    if not top_tickers:
        print("⚠️  No tickers found. Make sure you have articles with tickers in the database.")
    else:
        print(f"Found {len(top_tickers)} tickers\n")
        
        # Show top 10
        print("Top 10:")
        for i, (ticker, count) in enumerate(top_tickers[:10], 1):
            print(f"{i:2d}. {ticker:6s} - {count:4d} articles")
        
        print(f"\n... and {len(top_tickers) - 10} more\n")
        
        # Store for later use
        ticker_symbols = [ticker for ticker, _ in top_tickers]
        print(f"✅ Retrieved {len(ticker_symbols)} ticker symbols")
finally:
    db.close()


📊 Top 50 Tickers (last 24 hours):

Found 50 tickers

Top 10:
 1. PATH   -  390 articles
 2. AMD    -  283 articles
 3. SPY    -  277 articles
 4. POET   -  256 articles
 5. NVDA   -  169 articles
 6. GLD    -  129 articles
 7. RDDT   -   87 articles
 8. TLRY   -   82 articles
 9. NBIS   -   81 articles
10. GOOG   -   61 articles

... and 40 more

✅ Retrieved 50 ticker symbols


## 2. Test: Fetch Stock Price for a Single Ticker

Test the Yahoo Finance API integration by fetching price for a single stock.

In [ ]:
async def test_single_stock_price(symbol: str):
    """
    Test fetching price for a single stock.
    """
    stock_service = StockDataService()
    
    print(f"\n🔍 Fetching price for {symbol}...\n")
    
    data = await stock_service.get_stock_price(symbol)
    
    if data:
        print("✅ Successfully fetched data:")
        print(f"   Symbol: {data.get('symbol')}")
        print(f"   Price: ${data.get('price')}")
        print(f"   Previous Close: ${data.get('previous_close')}")
        print(f"   Change: ${data.get('change')} ({data.get('change_percent')}%)")
        print(f"   Market State: {data.get('market_state')}")
        print(f"   Exchange: {data.get('exchange')}")
        print(f"   Currency: {data.get('currency')}")
        return data
    else:
        print(f"❌ Failed to fetch data for {symbol}")
        return None

# Test with a well-known ticker
test_symbol = "AAPL"  # Apple
price_data = await test_single_stock_price(test_symbol)

In [3]:
    stock_service = StockDataService()


## 3. Test: Price Data Validation

Test the validation logic to ensure we don't store invalid prices.

In [4]:
data = await stock_service.get_stock_price("SPY")
data


{'symbol': 'SPY',
 'price': 671.08,
 'previous_close': 673.78,
 'change': -2.7,
 'change_percent': -0.4,
 'currency': 'USD',
 'market_state': 'POST',
 'exchange': 'PCX',
 'last_updated': '2025-10-09T23:18:54.500874'}

In [5]:
def validate_price_data(data: dict) -> bool:
    """
    Validate price data to ensure it's usable.
    """
    if not data:
        return False

    price = data.get("price")

    if price is None or price <= 0:
        print(f"❌ Invalid price: {price}")
        return False

    if price > 1000000:  # Sanity check
        print(f"❌ Unrealistically high price: ${price:,.2f}")
        return False

    return True

# Test validation with different scenarios
print("\n🧪 Testing Price Validation:\n")

test_cases = [
    {"name": "Valid price", "data": {"price": 150.25, "symbol": "AAPL"}, "expected": True},
    {"name": "Zero price", "data": {"price": 0, "symbol": "TEST"}, "expected": False},
    {"name": "Negative price", "data": {"price": -10, "symbol": "TEST"}, "expected": False},
    {"name": "None price", "data": {"price": None, "symbol": "TEST"}, "expected": False},
    {"name": "Too high price", "data": {"price": 2000000, "symbol": "TEST"}, "expected": False},
    {"name": "Empty data", "data": None, "expected": False},
]

for test in test_cases:
    result = validate_price_data(test["data"])
    status = "✅" if result == test["expected"] else "❌"
    print(f"{status} {test['name']:20s} - Expected: {test['expected']}, Got: {result}")

print("\n✅ Validation tests complete")


🧪 Testing Price Validation:

✅ Valid price          - Expected: True, Got: True
❌ Invalid price: 0
✅ Zero price           - Expected: False, Got: False
❌ Invalid price: -10
✅ Negative price       - Expected: False, Got: False
❌ Invalid price: None
✅ None price           - Expected: False, Got: False
❌ Unrealistically high price: $2,000,000.00
✅ Too high price       - Expected: False, Got: False
✅ Empty data           - Expected: False, Got: False

✅ Validation tests complete


## 4. Test: Fetch Multiple Stock Prices

Test fetching prices for multiple stocks (simulating the batch processing).

In [ ]:
async def test_multiple_stock_prices(symbols: list[str], limit: int = 5):
    """
    Test fetching prices for multiple stocks.
    """
    stock_service = StockDataService()
    
    # Limit to avoid rate limiting
    symbols = symbols[:limit]
    
    print(f"\n📊 Fetching prices for {len(symbols)} stocks...\n")
    
    results = []
    success_count = 0
    failed_count = 0
    
    for i, symbol in enumerate(symbols, 1):
        print(f"[{i}/{len(symbols)}] Fetching {symbol}...", end=" ")
        
        try:
            data = await stock_service.get_stock_price(symbol)
            
            if data and validate_price_data(data):
                print(f"✅ ${data['price']}")
                results.append({"symbol": symbol, "data": data, "status": "success"})
                success_count += 1
            else:
                print("❌ Invalid data")
                results.append({"symbol": symbol, "data": None, "status": "failed"})
                failed_count += 1
        except Exception as e:
            print(f"❌ Error: {e}")
            results.append({"symbol": symbol, "data": None, "status": "error", "error": str(e)})
            failed_count += 1
    
    print(f"\n📈 Summary:")
    print(f"   Success: {success_count}/{len(symbols)}")
    print(f"   Failed: {failed_count}/{len(symbols)}")
    
    return results

# Test with common stocks
test_symbols = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA"]
batch_results = await test_multiple_stock_prices(test_symbols, limit=5)

## 5. Test: Database Update

Test updating the stock_price table with fetched data.

In [ ]:
def update_stock_price(db, symbol: str, data: dict) -> bool:
    """
    Update or create stock price record in database.
    """
    try:
        now = datetime.now(UTC)
        existing = db.query(StockPrice).filter(StockPrice.symbol == symbol).first()
        
        if existing:
            print(f"   Updating existing record for {symbol}")
            existing.price = data["price"]
            existing.previous_close = data.get("previous_close")
            existing.change = data.get("change")
            existing.change_percent = data.get("change_percent")
            existing.market_state = data.get("market_state")
            existing.currency = data.get("currency", "USD")
            existing.exchange = data.get("exchange")
            existing.updated_at = now
        else:
            print(f"   Creating new record for {symbol}")
            new_price = StockPrice(
                symbol=symbol,
                price=data["price"],
                previous_close=data.get("previous_close"),
                change=data.get("change"),
                change_percent=data.get("change_percent"),
                market_state=data.get("market_state"),
                currency=data.get("currency", "USD"),
                exchange=data.get("exchange"),
                updated_at=now,
            )
            db.add(new_price)
        
        db.commit()
        return True
    except Exception as e:
        print(f"   ❌ Error updating {symbol}: {e}")
        db.rollback()
        return False

# Test database update with our fetched data
db = SessionLocal()
try:
    print("\n💾 Testing Database Updates:\n")
    
    success_count = 0
    for result in batch_results:
        if result["status"] == "success":
            symbol = result["symbol"]
            data = result["data"]
            
            print(f"Updating {symbol}...")
            if update_stock_price(db, symbol, data):
                success_count += 1
                print(f"   ✅ Successfully updated")
    
    print(f"\n📊 Updated {success_count} records in database")
    
    # Verify the updates
    print("\n🔍 Verifying updates in database:\n")
    for result in batch_results:
        if result["status"] == "success":
            symbol = result["symbol"]
            record = db.query(StockPrice).filter(StockPrice.symbol == symbol).first()
            
            if record:
                print(f"✅ {symbol}: ${record.price} (updated: {record.updated_at})")
            else:
                print(f"❌ {symbol}: Not found in database")
finally:
    db.close()

## 6. Full Integration Test

Run the complete flow: get top tickers, fetch prices, validate, and store.

In [ ]:
async def run_full_integration_test(limit: int = 10):
    """
    Run complete stock price collection flow.
    """
    print("\n" + "="*80)
    print("FULL INTEGRATION TEST")
    print("="*80 + "\n")
    
    db = SessionLocal()
    stock_service = StockDataService()
    
    try:
        # Step 1: Get top tickers
        print("📊 Step 1: Getting top tickers...")
        top_tickers = get_top_n_tickers(db, n=limit, hours=24)
        symbols = [ticker for ticker, _ in top_tickers]
        
        if not symbols:
            print("❌ No tickers found. Using default test symbols.")
            symbols = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA"][:limit]
        
        print(f"✅ Found {len(symbols)} tickers: {', '.join(symbols[:5])}...\n")
        
        # Step 2: Fetch prices
        print("📈 Step 2: Fetching stock prices...")
        success_count = 0
        failed_count = 0
        
        for i, symbol in enumerate(symbols, 1):
            print(f"[{i}/{len(symbols)}] {symbol}...", end=" ")
            
            try:
                # Fetch price
                data = await stock_service.get_stock_price(symbol)
                
                # Validate
                if data and validate_price_data(data):
                    # Update database
                    if update_stock_price(db, symbol, data):
                        print(f"✅ ${data['price']}")
                        success_count += 1
                    else:
                        print("❌ DB update failed")
                        failed_count += 1
                else:
                    print("❌ Invalid data")
                    failed_count += 1
                    
            except Exception as e:
                print(f"❌ Error: {e}")
                failed_count += 1
        
        # Summary
        print("\n" + "="*80)
        print("SUMMARY")
        print("="*80)
        print(f"Total Processed: {len(symbols)}")
        print(f"✅ Success: {success_count}")
        print(f"❌ Failed: {failed_count}")
        print(f"Success Rate: {(success_count/len(symbols)*100):.1f}%")
        print("="*80 + "\n")
        
    finally:
        db.close()

# Run the full test with 10 tickers
await run_full_integration_test(limit=10)

## 7. Query Current Stock Prices from Database

Check what stock prices are currently in the database.

In [ ]:
db = SessionLocal()
try:
    print("\n📊 Current Stock Prices in Database:\n")
    
    # Get all stock prices, ordered by most recently updated
    prices = db.query(StockPrice).order_by(StockPrice.updated_at.desc()).limit(20).all()
    
    if not prices:
        print("⚠️  No stock prices in database yet")
    else:
        print(f"Showing {len(prices)} most recently updated prices:\n")
        print(f"{'Symbol':8s} {'Price':>10s} {'Change':>10s} {'Market':>10s} {'Updated':>20s}")
        print("-" * 65)
        
        for price in prices:
            change_str = f"{price.change_percent:+.2f}%" if price.change_percent else "N/A"
            updated_str = price.updated_at.strftime("%Y-%m-%d %H:%M:%S") if price.updated_at else "N/A"
            market_str = price.market_state or "N/A"
            
            print(f"{price.symbol:8s} ${price.price:>9.2f} {change_str:>10s} {market_str:>10s} {updated_str:>20s}")
finally:
    db.close()

## 8. Test Specific Stock Query

Test querying a specific stock price from the database.

In [ ]:
def get_stock_price_from_db(symbol: str):
    """
    Get stock price for a specific symbol from database.
    """
    db = SessionLocal()
    try:
        price = db.query(StockPrice).filter(StockPrice.symbol == symbol.upper()).first()
        
        if price:
            print(f"\n✅ Found {symbol} in database:\n")
            print(f"   Symbol: {price.symbol}")
            print(f"   Price: ${price.price}")
            print(f"   Previous Close: ${price.previous_close}")
            print(f"   Change: ${price.change} ({price.change_percent:+.2f}%)")
            print(f"   Market State: {price.market_state}")
            print(f"   Exchange: {price.exchange}")
            print(f"   Last Updated: {price.updated_at}")
            
            # Check if stale (> 30 minutes)
            age = datetime.now(UTC) - price.updated_at
            if age.total_seconds() > 1800:  # 30 minutes
                print(f"\n   ⚠️  Price is stale (age: {age})")
            else:
                print(f"\n   ✅ Price is fresh (age: {age})")
            
            return price
        else:
            print(f"\n❌ {symbol} not found in database")
            return None
    finally:
        db.close()

# Test querying specific stocks
test_queries = ["AAPL", "TSLA", "MSFT"]
for symbol in test_queries:
    get_stock_price_from_db(symbol)

## Summary

This notebook tested all the major components of the stock price collector:

1. ✅ Getting top 50 tickers from article mentions
2. ✅ Fetching stock prices from Yahoo Finance
3. ✅ Validating price data
4. ✅ Batch processing multiple stocks
5. ✅ Updating database with new prices
6. ✅ Full integration test
7. ✅ Querying prices from database

The actual job ([jobs/jobs/stock_price_collector.py](../jobs/jobs/stock_price_collector.py)) combines all these functions into a single automated process that runs every 15 minutes.